In [1]:
import os
print(os.getcwd())

import sys
sys.path.append(r'C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush\Scripts\GANalyze\pytorch\assessors')

from emonet import EmoNet, emonet

# Example usage of EmoNet
model, input_transform, output_transform = emonet(tencrop=False)
print("Model loaded successfully.")


C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\scripts


C:\Users\sOrOush\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\sOrOush\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush\Scripts\GANalyze\pytorch\assessors\emonet.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blo

Model loaded successfully.


In [2]:
import os
import sys
import torch
import numpy as np
import socket
import argparse
import json
import subprocess
from contextlib import contextmanager
import torch.distributed as dist
from torch.nn.parallel.distributed import DistributedDataParallel
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as T
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt

sub = 1
bs = 30

In [3]:
# Cell 2

import sys
sys.path.append(r'C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush\Scripts\GANalyze\pytorch\assessors')

import emonet

assessor_in = 'emonet'
assessor_elements = getattr(emonet, assessor_in)(False)
if isinstance(assessor_elements, tuple):
    assessor = assessor_elements[0]
    input_transform = assessor_elements[1]
    output_transform = assessor_elements[2]
else:
    assessor = assessor_elements

    def input_transform(x):
        return x  # identity, no preprocessing

    def output_transform(x):
        return x  # identity, no postprocessing

if hasattr(assessor, 'parameters'):
    for p in assessor.parameters():
        p.requires_grad = False
        assessor.eval()
        assessor.to()

C:\Users\sOrOush\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\sOrOush\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush\Scripts\GANalyze\pytorch\assessors\emonet.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blo

In [4]:
def test_images_prep_assessor(data_path):
    data = np.load(data_path).astype(np.uint8)
    dim0, dim1, dim2, dim3 = data.shape[0], 256, 256, 3
    data_tensor = torch.empty(dim0, dim3, dim1, dim2)

    for idx in range(len(data)):
            img = Image.fromarray(data[idx])
            img = T.functional.resize(img, (256, 256))
            img_tensor = T.functional.to_tensor(img)  # Automatically converts to float and scales to [0, 1]

            # Store the normalized image tensor
            data_tensor[idx] = img_tensor

    return data_tensor

In [5]:
import re
def natural_sort_key(s):
    return [int(c) if c.isdigit() else c.lower() for c in re.split('(\d+)', s)]

def test_images_prep_assessor_CLIP(image_folder, target_size=(256, 256)):
    data_tensor = []

    # Get all image files and sort them
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    image_files.sort(key=natural_sort_key)

    for filename in image_files:
        img_path = os.path.join(image_folder, filename)
        img = Image.open(img_path).convert('RGB')

        transform = T.Compose([
            T.Resize(target_size),
            T.ToTensor()  # This converts to float and scales to [0, 1]
        ])

        img_tensor = transform(img)
        data_tensor.append(img_tensor)

    return torch.stack(data_tensor)

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sOrOush\AppData\Local\Temp\ipykernel_42324\1355969087.py:3: SyntaxWarning: invalid escape sequence '\d'
  return [int(c) if c.isdigit() else c.lower() for c in re.split('(\d+)', s)]


In [6]:
emonet_res = {
    'nsdgeneral_degree1': [],
    'nsdgeneral_degree2': [],
    'nsdgeneral_degree3': [],
    'nsdgeneral_degree4': [],
    'Default_degree1': [],
    'Default_degree2': [],
    'Default_degree3': [],
    'Default_degree4': [],
    'Auditory_degree1': [],
    'Auditory_degree2': [],
    'Auditory_degree3': [],
    'Auditory_degree4': []
}

In [7]:
## real_img
image_path = r"C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\00_Original_Images"
test_tensor = test_images_prep_assessor_CLIP(image_path)
# Reshape while keeping the last 3 dimensions
test_im = test_tensor.view(-1, *test_tensor.shape[-3:])

#------  Apply the transformations and assess
score = assessor(test_im)
emonet_res['real_img'] = [s.detach().numpy()[0] for s in score]
print("====Real Done!====")


====Real Done!====


In [8]:
emonet_res.keys()

dict_keys(['nsdgeneral_degree1', 'nsdgeneral_degree2', 'nsdgeneral_degree3', 'nsdgeneral_degree4', 'Default_degree1', 'Default_degree2', 'Default_degree3', 'Default_degree4', 'Auditory_degree1', 'Auditory_degree2', 'Auditory_degree3', 'Auditory_degree4', 'real_img'])

In [9]:
emonet_res_keys = list(emonet_res.keys())

In [10]:

# Define the base directory where the "VDVAE" results are stored
base_dir = r"C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\generated images\VDVAE"


# Iterate over groups and degrees to build keys and load images
for group in groups:
    for degree in degrees:
        # Construct a unique key, e.g., "nsdgeneral_degree1"
        key = f"{group}_degree{degree}"
        # base_dir/<group>/degree<degree>/subj01
        image_path = os.path.join(base_dir, group, f"degree{degree}", "subj01")
        test_tensor = test_images_prep_assessor_CLIP(image_path)
        test_im = test_tensor.view(-1, *test_tensor.shape[-3:])
        score = assessor(test_im)
        emonet_res[key] = [s.detach().numpy()[0] for s in score]
        print(f"{key} is done.")


nsdgeneral_degree1 is done.
nsdgeneral_degree2 is done.
nsdgeneral_degree3 is done.
nsdgeneral_degree4 is done.
Default_degree1 is done.
Default_degree2 is done.
Default_degree3 is done.
Default_degree4 is done.
Auditory_degree1 is done.
Auditory_degree2 is done.
Auditory_degree3 is done.
Auditory_degree4 is done.


In [11]:
import pickle

# Assuming your dictionary is stored in 'emonet_res'
file_path = r'C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\assessor_results\emonet_res.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(emonet_res, file)
